We will collect text data using Twitter API.

In [1]:
!pip install tweepy

  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)


In [2]:
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

# Initializing Twitter API

In [4]:
consumer_key ="blNt9nEmE48NaD1rEk7tf73Xh"
consumer_secret ="tu2BIF4XiRtvl24uGA7MY9a7r3sRL3k3jcFD8jSo4H90uoGNkJ"
access_token ="1360881263679135745-w3pT5CLx2uCf7D7F1IfHAvzW6AgOYc"
access_token_secret ="FH1Q8OsUWgPDZV3tJxyuR49nIictTEqX0W1VYWxUWntJu"

In [5]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [6]:
query = "Narendra Modi Prime Minister"

In [7]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

# for tweet in Tweets:
#     print(tweet,"\n")
#     print(".....................................\n")

## Creating Dataframe by fetching data using tweepy

In [8]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [9]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [10]:
stream(data=[query] , file_name='my_tweets')

In [11]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @Satirical_Dhruv: Without any degree I am a...,Raju K Pahuchungia,13470,892,India,False,0,626,2021-05-27 13:59:14
1,RT @Satirical_Dhruv: Without any degree I am a...,@UnitedIndians,41640,261,,False,0,626,2021-05-27 13:58:37
2,RT @Satirical_Dhruv: Without any degree I am a...,Vishal Dsouza,1717,7,"Al Ain, United Arab Emirates",False,0,626,2021-05-27 13:58:08
3,@DevinderSolanki @hansrajhansHRH @BJP4Delhi @s...,Satbir Solanki,4003,149,कृष्ण विहार दिल्ली 86,False,0,0,2021-05-27 13:57:57
4,RT @Satirical_Dhruv: Without any degree I am a...,Saadiya ahmed,8381,263,Under The Sky,False,0,626,2021-05-27 13:57:51


In [12]:
!pip install textblob

## Cleaning Tweets and analyzing using TextBlob

In [13]:
from textblob import TextBlob

In [14]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [15]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [16]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [17]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,RT @Satirical_Dhruv: Without any degree I am a...,Raju K Pahuchungia,13470,892,India,False,0,626,2021-05-27 13:59:14,RT Dhruv Without any degree I am a doctor Baba...
1,RT @Satirical_Dhruv: Without any degree I am a...,@UnitedIndians,41640,261,,False,0,626,2021-05-27 13:58:37,RT Dhruv Without any degree I am a doctor Baba...
2,RT @Satirical_Dhruv: Without any degree I am a...,Vishal Dsouza,1717,7,"Al Ain, United Arab Emirates",False,0,626,2021-05-27 13:58:08,RT Dhruv Without any degree I am a doctor Baba...
3,@DevinderSolanki @hansrajhansHRH @BJP4Delhi @s...,Satbir Solanki,4003,149,कृष्ण विहार दिल्ली 86,False,0,0,2021-05-27 13:57:57,Prime Minister of India India s Prime Minister...
4,RT @Satirical_Dhruv: Without any degree I am a...,Saadiya ahmed,8381,263,Under The Sky,False,0,626,2021-05-27 13:57:51,RT Dhruv Without any degree I am a doctor Baba...


In [18]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @Satirical_Dhruv: Without any degree I am a...,Raju K Pahuchungia,13470,892,India,False,0,626,2021-05-27 13:59:14,RT Dhruv Without any degree I am a doctor Baba...,Neutral
1,RT @Satirical_Dhruv: Without any degree I am a...,@UnitedIndians,41640,261,,False,0,626,2021-05-27 13:58:37,RT Dhruv Without any degree I am a doctor Baba...,Neutral
2,RT @Satirical_Dhruv: Without any degree I am a...,Vishal Dsouza,1717,7,"Al Ain, United Arab Emirates",False,0,626,2021-05-27 13:58:08,RT Dhruv Without any degree I am a doctor Baba...,Neutral
3,@DevinderSolanki @hansrajhansHRH @BJP4Delhi @s...,Satbir Solanki,4003,149,कृष्ण विहार दिल्ली 86,False,0,0,2021-05-27 13:57:57,Prime Minister of India India s Prime Minister...,Neutral
4,RT @Satirical_Dhruv: Without any degree I am a...,Saadiya ahmed,8381,263,Under The Sky,False,0,626,2021-05-27 13:57:51,RT Dhruv Without any degree I am a doctor Baba...,Neutral


In [19]:
df[df['User_verified'] == True].count()

Tweets                 8
User                   8
User_statuses_count    8
user_followers         8
User_location          8
User_verified          8
fav_count              8
rt_count               8
tweet_date             8
clean_tweet            8
Sentiment              8
dtype: int64

In [20]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @Satirical_Dhruv: Without any degree I am a doctor - Baba Ramdev

Without any degree I am union minister - Smriti Irani

Without any deg…

Clean tweet:
 RT Dhruv Without any degree I am a doctor Baba Ramdev Without any degree I am union minister Smriti Irani Without any deg

Sentiment of the tweet:
 Neutral


In [22]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @Satirical_Dhruv: Without any degree I am a doctor - Baba Ramdev

Without any degree I am union minister - Smriti Irani

Without any deg…

Clean tweet:
 RT Dhruv Without any degree I am a doctor Baba Ramdev Without any degree I am union minister Smriti Irani Without any deg

Sentiment of the tweet:
 Neutral


In [23]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @PaulEDawson: “If more mining happens everything will change; the natural resources will be gone, our way of life will disappear, everyt…

Clean tweet:
 RT If more mining happens everything will change the natural resources will be gone our way of life will disappear everyt

Sentiment of the tweet:
 Positive


In [24]:
df[df.Sentiment == 'Positive'].shape[0]

177

In [25]:
df[df.Sentiment == 'Neutral'].shape[0]

815

In [26]:
df[df.Sentiment == 'Negative'].shape[0]

8